# Pandas Pipe pre work

I downloaded a csv file about 80 cereal products with their dietary characteristics on <a href="https://perso.telecom-paristech.fr/eagan/class/igr204/datasets"> Project datasets.</a> The data is seperated by semicolons instead of commas. Pnadas handles it when you indicate what the seperator is by using `sep`. 

In [1]:
import pandas as pd

df = pd.read_csv('Data/cereal.csv', sep=';')
df

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,String,Categorical,Categorical,Int,Int,Int,Int,Float,Float,Int,Int,Int,Int,Float,Float,Float
1,100% Bran,N,C,70,4,1,130,10,5,6,280,25,3,1,0.33,68.402973
2,100% Natural Bran,Q,C,120,3,5,15,2,8,8,135,0,3,1,1,33.983679
3,All-Bran,K,C,70,4,1,260,9,7,5,320,25,3,1,0.33,59.425505
4,All-Bran with Extra Fiber,K,C,50,4,0,140,14,8,0,330,25,3,1,0.5,93.704912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,Triples,G,C,110,2,1,250,0,21,3,60,25,3,1,0.75,39.106174
74,Trix,G,C,110,1,1,140,0,13,12,25,25,2,1,1,27.753301
75,Wheat Chex,R,C,100,3,1,230,3,17,3,115,25,1,1,0.67,49.787445
76,Wheaties,G,C,100,3,1,200,3,17,3,110,25,1,1,1,51.592193


Here is the goal of this data analysis.

1. How many kinds of cereal each manufacturer has.
2. We want to find the best cereal for Sam. Sam's cereal should be protein >= 2, fat <=2, sugars <=7. Sam wants low calories and good rating as well. 

Let's start. I do not want to change anything in the original data frame. So I will work with its copy. In this notebook, I will work with `pipe`s so that it is easy to find out what I have done in each code. This is also good when I want to redo something.

In [2]:
def copying(data):
    return data.copy()

The first row looks unnecessary. So we remove it.

In [3]:
def dropping(data):
    return data.drop([0])

Let's see if it works.

In [4]:
df.pipe(copying).pipe(dropping).head()

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
1,100% Bran,N,C,70,4,1,130,10,5,6,280,25,3,1,0.33,68.402973
2,100% Natural Bran,Q,C,120,3,5,15,2,8,8,135,0,3,1,1,33.983679
3,All-Bran,K,C,70,4,1,260,9,7,5,320,25,3,1,0.33,59.425505
4,All-Bran with Extra Fiber,K,C,50,4,0,140,14,8,0,330,25,3,1,0.5,93.704912
5,Almond Delight,R,C,110,2,2,200,1,14,8,-1,25,3,1,0.75,34.384843


What data type is each column?

In [5]:
df.dtypes

name        object
mfr         object
type        object
calories    object
protein     object
fat         object
sodium      object
fiber       object
carbo       object
sugars      object
potass      object
vitamins    object
shelf       object
weight      object
cups        object
rating      object
dtype: object

We want to change all the columns except first three into numbers. What is the size of this data frame and how many columns are there?

In [6]:
df.shape

(78, 16)

Let's convert last 13 columns into numbers. Maybe Pandas can magically do it? 

In [7]:
def data_types(data):
    return data.convert_dtypes().dtypes   
 
df.pipe(copying).pipe(dropping).pipe(data_types)

name        string
mfr         string
type        string
calories    string
protein     string
fat         string
sodium      string
fiber       string
carbo       string
sugars      string
potass      string
vitamins    string
shelf       string
weight      string
cups        string
rating      string
dtype: object

No. Pandas can't do it.
We will make a list of column names and then use `apply(pd.to_numeric)`. Is there a better way to convert 13 columns into numbers at once?

In [8]:
cols = list(df.pipe(copying).pipe(dropping).columns)[3:15]
print(cols)

['calories', 'protein', 'fat', 'sodium', 'fiber', 'carbo', 'sugars', 'potass', 'vitamins', 'shelf', 'weight', 'cups']


OK. We change all columns in cols into numbers.

In [9]:
def new_data_types(data):
    data[cols] = data[cols].apply(pd.to_numeric)
    return data   

df.pipe(copying).pipe(dropping).pipe(new_data_types).dtypes

name         object
mfr          object
type         object
calories      int64
protein       int64
fat           int64
sodium        int64
fiber       float64
carbo       float64
sugars        int64
potass        int64
vitamins      int64
shelf         int64
weight      float64
cups        float64
rating       object
dtype: object

### 1. We want to know how many products each manufacturer has.

In [10]:
def mfr_group(data):
    gdata = data.groupby(['mfr'])
    return data['mfr'].value_counts()
 
df.pipe(copying).pipe(dropping).pipe(mfr_group)

K    23
G    22
P     9
R     8
Q     8
N     6
A     1
Name: mfr, dtype: int64

When we are only interested in mfr which has more than 10 products, we can use `groupby` and `agg`. 

In [11]:
def removing_low_mfr(data, min_products = 10):
    mfr_no_products = (data
                 .groupby('mfr')
                 .agg(n=('name', 'count'))
                 .loc[lambda d: d['n'] >= min_products]
                 .index)
    return (data
            .loc[lambda d: d['mfr'].isin(mfr_no_products)])

df.pipe(copying).pipe(dropping).pipe(removing_low_mfr)

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
3,All-Bran,K,C,70,4,1,260,9,7,5,320,25,3,1,0.33,59.425505
4,All-Bran with Extra Fiber,K,C,50,4,0,140,14,8,0,330,25,3,1,0.5,93.704912
6,Apple Cinnamon Cheerios,G,C,110,2,2,180,1.5,10.5,10,70,25,1,1,0.75,29.509541
7,Apple Jacks,K,C,110,2,0,125,1,11,14,30,25,2,1,1,33.174094
8,Basic 4,G,C,130,3,2,210,2,18,8,100,25,3,1.33,0.75,37.038562
12,Cheerios,G,C,110,6,2,290,2,17,1,105,25,1,1,1.25,50.764999
13,Cinnamon Toast Crunch,G,C,120,1,3,210,0,13,9,45,25,2,1,0.75,19.823573
14,Clusters,G,C,110,3,2,140,2,13,7,105,25,3,1,0.5,40.400208
15,Cocoa Puffs,G,C,110,1,1,180,0,12,13,55,25,2,1,1,22.736446
17,Corn Flakes,K,C,100,2,0,290,1,21,2,35,25,1,1,1,45.863324


We can also make a multiindex. I have no idea what type is in the column names. Anyway, let mfr, type and name be indexes.

In [12]:
def indexing(data):
    return data.set_index(['type', 'mfr', 'name']).sort_index()

Let's have a look!

In [13]:
clean_df = df.pipe(copying).pipe(dropping).pipe(indexing)
clean_df

calories protein fat sodium fiber carbo  \
type mfr name                                                              
C    G   Apple Cinnamon Cheerios      110       2   2    180   1.5  10.5   
         Basic 4                      130       3   2    210     2    18   
         Cheerios                     110       6   2    290     2    17   
         Cinnamon Toast Crunch        120       1   3    210     0    13   
         Clusters                     110       3   2    140     2    13   
...                                   ...     ...  ..    ...   ...   ...   
     R   Rice Chex                    110       1   0    240     0    23   
         Wheat Chex                   100       3   1    230     3    17   
H    A   Maypo                        100       4   1      0     0    16   
     N   Cream of Wheat (Quick)       100       3   0     80     1    21   
     Q   Quaker Oatmeal               100       5   2      0   2.7    -1   

                                 sugars potass vitamins shelf weight  cups  \
type mfr name                                                                
C    G   Apple Cinnamon Cheerios     10     70       25     1      1  0.75   
         Basic 4                      8    100       25     3   1.33  0.75   
         Cheerios                     1    105       25     1      1  1.25   
         Cinnamon Toast Crunch        9     45       25     2      1  0.75   
         Clusters                     7    105       25     3      1   0.5   
...                                 ...    ...      ...   ...    ...   ...   
     R   Rice Chex                    2     30       25     1      1  1.13   
         Wheat Chex                   3    115       25     1      1  0.67   
H    A   Maypo                        3     95       25     2      1     1   
     N   Cream of Wheat (Quick)       0     -1        0     2      1     1   
     Q   Quaker Oatmeal              -1    110        0     1      1  0.67   

                                     rating  
type mfr name                                
C    G   Apple Cinnamon Cheerios  29.509541  
         Basic 4                  37.038562  
         Cheerios                 50.764999  
         Cinnamon Toast Crunch    19.823573  
         Clusters                 40.400208  
...                                     ...  
     R   Rice Chex                41.998933  
         Wheat Chex               49.787445  
H    A   Maypo                    54.850917  
     N   Cream of Wheat (Quick)   64.533816  
     Q   Quaker Oatmeal           50.828392  

[77 rows x 13 columns]

If we are only interested in manufacturer G and K, then we can use `IndexSlice`.

In [14]:
clean_df.loc[pd.IndexSlice[:, ['G', 'K']], :]

calories protein fat sodium fiber carbo  \
type mfr name                                                                  
C    G   Apple Cinnamon Cheerios          110       2   2    180   1.5  10.5   
         Basic 4                          130       3   2    210     2    18   
         Cheerios                         110       6   2    290     2    17   
         Cinnamon Toast Crunch            120       1   3    210     0    13   
         Clusters                         110       3   2    140     2    13   
         Cocoa Puffs                      110       1   1    180     0    12   
         Count Chocula                    110       1   1    180     0    12   
         Crispy Wheat & Raisins           100       2   1    140     2    11   
         Golden Grahams                   110       1   1    280     0    15   
         Honey Nut Cheerios               110       3   1    250   1.5  11.5   
         Kix                              110       2   1    260     0    21   
         Lucky Charms                     110       2   1    180     0    12   
         Multi-Grain Cheerios             100       2   1    220     2    15   
         Oatmeal Raisin Crisp             130       3   2    170   1.5  13.5   
         Raisin Nut Bran                  100       3   2    140   2.5  10.5   
         Total Corn Flakes                110       2   1    200     0    21   
         Total Raisin Bran                140       3   1    190     4    15   
         Total Whole Grain                100       3   1    200     3    16   
         Triples                          110       2   1    250     0    21   
         Trix                             110       1   1    140     0    13   
         Wheaties                         100       3   1    200     3    17   
         Wheaties Honey Gold              110       2   1    200     1    16   
     K   All-Bran                          70       4   1    260     9     7   
         All-Bran with Extra Fiber         50       4   0    140    14     8   
         Apple Jacks                      110       2   0    125     1    11   
         Corn Flakes                      100       2   0    290     1    21   
         Corn Pops                        110       1   0     90     1    13   
         Cracklin' Oat Bran               110       3   3    140     4    10   
         Crispix                          110       2   0    220     1    21   
         Froot Loops                      110       2   1    125     1    11   
         Frosted Flakes                   110       1   0    200     1    14   
         Frosted Mini-Wheats              100       3   0      0     3    14   
         Fruitful Bran                    120       3   0    240     5    14   
         Just Right Crunchy  Nuggets      110       2   1    170     1    17   
         Just Right Fruit & Nut           140       3   1    170     2    20   
         Mueslix Crispy Blend             160       3   2    150     3    17   
         Nut&Honey Crunch                 120       2   1    190     0    15   
         Nutri-Grain Almond-Raisin        140       3   2    220     3    21   
         Nutri-grain Wheat                 90       3   0    170     3    18   
         Product 19                       100       3   0    320     1    20   
         Raisin Bran                      120       3   1    210     5    14   
         Raisin Squares                    90       2   0      0     2    15   
         Rice Krispies                    110       2   0    290     0    22   
         Smacks                           110       2   1     70     1     9   
         Special K                        110       6   0    230     1    16   

                                     sugars potass vitamins shelf weight  \
type mfr name                                                              
C    G   Apple Cinnamon Cheerios         10     70       25     1      1   
         Basic 4                          8    1

### 2. We want to find the best cereal for Sam.
Let's find cereals which have protein >= 2, fat <=2, sugars <=7. 

In [15]:
def nutrition_sieve(data, min_protein = 2, max_fat=2, max_sugars=7):
    mask = (data['protein'] >=min_protein) & (data['fat'] <= max_fat) & (data['sugars'] <= max_sugars)
    return data[mask]
df.pipe(copying).pipe(dropping).pipe(new_data_types).pipe(indexing).pipe(nutrition_sieve)

calories  protein  fat  sodium  fiber  \
type mfr name                                                                 
C    G   Cheerios                          110        6    2     290    2.0   
         Clusters                          110        3    2     140    2.0   
         Kix                               110        2    1     260    0.0   
         Multi-Grain Cheerios              100        2    1     220    2.0   
         Total Corn Flakes                 110        2    1     200    0.0   
         Total Whole Grain                 100        3    1     200    3.0   
         Triples                           110        2    1     250    0.0   
         Wheaties                          100        3    1     200    3.0   
     K   All-Bran                           70        4    1     260    9.0   
         All-Bran with Extra Fiber          50        4    0     140   14.0   
         Corn Flakes                       100        2    0     290    1.0   
         Crispix                           110        2    0     220    1.0   
         Frosted Mini-Wheats               100        3    0       0    3.0   
         Just Right Crunchy  Nuggets       110        2    1     170    1.0   
         Nutri-Grain Almond-Raisin         140        3    2     220    3.0   
         Nutri-grain Wheat                  90        3    0     170    3.0   
         Product 19                        100        3    0     320    1.0   
         Raisin Squares                     90        2    0       0    2.0   
         Rice Krispies                     110        2    0     290    0.0   
         Special K                         110        6    0     230    1.0   
     N   100% Bran                          70        4    1     130   10.0   
         Shredded Wheat                     80        2    0       0    3.0   
         Shredded Wheat 'n'Bran             90        3    0       0    4.0   
         Shredded Wheat spoon size          90        3    0       0    3.0   
         Strawberry Fruit Wheats            90        2    0      15    3.0   
     P   Bran Flakes                        90        3    0     210    5.0   
         Grape Nuts Flakes                 100        3    1     140    3.0   
         Grape-Nuts                        110        3    0     170    3.0   
     Q   Life                              100        4    2     150    2.0   
         Puffed Wheat                       50        2    0       0    1.0   
         Quaker Oat Squares                100        4    1     135    2.0   
     R   Bran Chex                          90        2    1     200    4.0   
         Corn Chex                         110        2    0     280    0.0   
         Double Chex                       100        2    0     190    1.0   
         Wheat Chex                        100        3    1     230    3.0   
H    A   Maypo                             100        4    1       0    0.0   
     N   Cream of Wheat (Quick)            100        3    0      80    1.0   
     Q   Quaker Oatmeal                    100        5    2       0    2.7   

                                      carbo  sugars  potass  vitamins  shelf  \
type mfr name                                                                  
C    G   Cheerios                      17.0       1     105        25      1   
         Clusters                      13.0       7     105        25      3   
         Kix                           21.0       3      40        25      2   
         Multi-Grain Cheerios          15.0       6      90        25      1   
         Total Corn Flakes             21.0       3      35       100      3   
         Total Whole Grain             16.0       3     110       100      3   
         Triples                       21.0       3      60        25      3   
         Wheaties                      17.0       3     110        25      1   
     K   All-Bran                       7.0       5     320        25      3 

We want to find the best cereal forSam wants low calories and good rating as well. 

In [16]:
def sorting(data):
    data1 = data.sort_values(['rating'], ascending=False)
    return data1.sort_values(['calories'])
df.pipe(copying).pipe(dropping).pipe(new_data_types).pipe(indexing).pipe(nutrition_sieve).pipe(sorting)

calories  protein  fat  sodium  fiber  \
type mfr name                                                                 
C    K   All-Bran with Extra Fiber          50        4    0     140   14.0   
     Q   Puffed Wheat                       50        2    0       0    1.0   
     N   100% Bran                          70        4    1     130   10.0   
     K   All-Bran                           70        4    1     260    9.0   
     N   Shredded Wheat                     80        2    0       0    3.0   
     R   Bran Chex                          90        2    1     200    4.0   
     P   Bran Flakes                        90        3    0     210    5.0   
     K   Raisin Squares                     90        2    0       0    2.0   
     N   Strawberry Fruit Wheats            90        2    0      15    3.0   
         Shredded Wheat spoon size          90        3    0       0    3.0   
         Shredded Wheat 'n'Bran             90        3    0       0    4.0   
     K   Nutri-grain Wheat                  90        3    0     170    3.0   
     Q   Quaker Oat Squares                100        4    1     135    2.0   
     K   Product 19                        100        3    0     320    1.0   
     G   Multi-Grain Cheerios              100        2    1     220    2.0   
     R   Double Chex                       100        2    0     190    1.0   
     Q   Life                              100        4    2     150    2.0   
     K   Corn Flakes                       100        2    0     290    1.0   
     G   Total Whole Grain                 100        3    1     200    3.0   
     R   Wheat Chex                        100        3    1     230    3.0   
H    Q   Quaker Oatmeal                    100        5    2       0    2.7   
C    G   Wheaties                          100        3    1     200    3.0   
     P   Grape Nuts Flakes                 100        3    1     140    3.0   
H    N   Cream of Wheat (Quick)            100        3    0      80    1.0   
     A   Maypo                             100        4    1       0    0.0   
C    K   Frosted Mini-Wheats               100        3    0       0    3.0   
     G   Total Corn Flakes                 110        2    1     200    0.0   
         Triples                           110        2    1     250    0.0   
         Kix                               110        2    1     260    0.0   
         Clusters                          110        3    2     140    2.0   
     K   Rice Krispies                     110        2    0     290    0.0   
     R   Corn Chex                         110        2    0     280    0.0   
     G   Cheerios                          110        6    2     290    2.0   
     P   Grape-Nuts                        110        3    0     170    3.0   
     K   Special K                         110        6    0     230    1.0   
         Crispix                           110        2    0     220    1.0   
         Just Right Crunchy  Nuggets       110        2    1     170    1.0   
         Nutri-Grain Almond-Raisin         140        3    2     220    3.0   

                                      carbo  sugars  potass  vitamins  shelf  \
type mfr name                                                                  
C    K   All-Bran with Extra Fiber      8.0       0     330        25      3   
     Q   Puffed Wheat                  10.0       0      50         0      3   
     N   100% Bran                      5.0       6     280        25      3   
     K   All-Bran                       7.0       5     320        25      3   
     N   Shredded Wheat                16.0       0      95         0      1   
     R   Bran Chex                     15.0       6     125        25      1   
     P   Bran Flakes                   13.0       5     190        25      3   
     K   Raisin Squares                15.0       6     110        25      3   
     N   Strawberry Fruit Wheats       15.0       5      90        25      2 

All-Bran with Extra Fiber will be the best cereal for Sam.	

To see the result, one should scroll down a lot. For people who only want to know the result, let's use inheritance. We will introduce logging and decorator in order to each pipe properly works. The final version of this data analysis is in the folder result.

## References

To study Pandas Pipe, I watched <a href="https://calmcode.io/pandas-pipe/introduction.html">pandas-pipe tutorial in calmcode</a> and <a href="https://tomaugspurger.github.io/modern-1-intro.html">modern pandas</a>.

To study logging and decorators I watched <a href="https://calmcode.io/logging/introduction.html"> logging tutorial in calmcode</a> and <a href="https://calmcode.io/decorators/introduction.html ">decorator tutorial in calmcode</a>.